In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import nnde
from mpl_toolkits.mplot3d import Axes3D
np.seterr(all='raise')
# import pixiedust

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

# Solving ODEs

## Example 1

$\frac{d}{dx}\Psi+(x+\frac{1+3x^2}{1+x+x^3})\Psi=x^3+2x+x^2\frac{1+3x^2}{1+x+x^3}$

With boundary initial condition $\Psi(0)=1$ and domain $x\in[0,1]$

In [2]:
search = lambda A, B: np.array([i for i in A.flatten().tolist() if i not in B.flatten().tolist()])
Xe1_interpolation = np.linspace(0,1,num=1000)
Xe1_interpolation = Xe1_interpolation.reshape((Xe1_interpolation.shape[0],1,1))
Xe1 = Xe1_interpolation[::100]
Xe1_interpolation = search(Xe1_interpolation, Xe1)
Xe1_interpolation = Xe1_interpolation.reshape((Xe1_interpolation.shape[0],1,1))

The trial solution for this case is $\Psi(x)=1 + x N(x)$.
The first function below is the function $A(x)=1$
and the second function is the function $B(x)=x$.

In [3]:
def example1_initial_value(point):
  return 1

In [4]:
def example1_boundary_vanishing(point):
  return point

### Defining the loss function for a single point and a whole set

The loss function is based on the formula:
$$Loss(N)=\sum_i \left(L\Psi(x_i, N(x_i))-f(x_i,\Psi(x_i, N(x_i))) \right)^2$$
Where N(x) is the neural network and L is some differential operator.

In [5]:
def example1_loss_function_single_point(self, point, non_squared=False, *kwargs):
  N = self.forward_pass(point, 0)
  dN = self.forward_pass(point, 1)
  loss = (
      point * dN + N + (point + (1 + 3 * point ** 2)/(1 + point + point ** 3)) * (1 + point * N) 
      - point ** 3 - 2 * point - point ** 2 *(1 + 3 * point ** 2)/(1 + point + point ** 3)
    )
  if not non_squared:
    loss = loss ** 2
  return loss[0,0]

In [6]:
def example1_loss_function(self, samples, *kwargs):
  loss = 0
  for i in range(samples.shape[0]):
    loss += self.loss_function_single_point(self, samples[i])
  return loss/samples.shape[0]

### Defining the update rules

The following functions represent $\frac{\partial Loss}{\partial \vec{b}}$, $\frac{\partial Loss}{\partial H}$, and $\frac{\partial Loss}{\partial V}$

In [7]:
def example1_bias_change(self, point, label, *kwargs):
  db = np.zeros((self.hidden_dim, 1)).astype(dtype="float64")
  loss_sqrt = self.loss_function_single_point(self, point, non_squared=True)
  db_N = self.network_derivative_bias(point, 0)
  db_DN = self.network_derivative_bias(point, 1)
  point = point.reshape((1,))
  for m in range(self.hidden_dim):
    db[m] += 2 * loss_sqrt * (
      point * db_DN[0, 0, m] + db_N[0, 0, m] + (point + (1 + 3 * point ** 2)/(1 + point + point ** 3)) * point * db_N[0, 0, m])
  return db

In [8]:
def example1_hidden_weights_change(self, point, *kwargs):
  dH = np.zeros((self.hidden_dim, self.input_dim)).astype(dtype="float64")
  loss_sqrt = self.loss_function_single_point(self, point, non_squared=True)
  dH_N = self.network_derivative_hidden_weights(point, 0)
  dH_DN = self.network_derivative_hidden_weights(point, 1)
  for m in range(self.hidden_dim):
    for p in range(self.input_dim):
      dH[m, p] += 2 * loss_sqrt * (
        point * dH_DN[0, 0, m, p] + dH_N[0, 0, m, p] + (point + (1 + 3 * point ** 2)/(1 + point + point ** 3)) * point * dH_N[0, 0, m, p])
  return dH

In [9]:
def example1_visible_weights_change(self, point, *kwargs):
  dV = np.zeros((self.visible_dim, self.hidden_dim)).astype(dtype="float64")
  loss_sqrt = self.loss_function_single_point(self, point, non_squared=True)
  dV_N = self.network_derivative_visible_weights(point, 0)
  dV_DN = self.network_derivative_visible_weights(point, 1)
  for m in range(self.visible_dim):
    for p in range(self.hidden_dim):
      dV[m, p] += 2 * loss_sqrt * (
        point * dV_DN[0, 0, m, p] + dV_N[0, 0, m, p] + (point + (1 + 3 * point ** 2)/(1 + point + point ** 3)) * point * dV_N[0, 0, m, p])
  return dV

### Defining the trial solution with an apropiate network

In [10]:
example1_trial_solution = nnde.TrialSolution(loss_function=example1_loss_function_single_point,
                                        boundary_condition_value_function=example1_initial_value,
                                        boundary_vanishing_function=example1_boundary_vanishing,
                                        input_dim=1, hidden_dim=10, output_dim=1, learning_rate=1e-1, momentum=1e-1)

### Training

In [11]:
example1_trial_solution.train(samples=Xe1, epochs=100, batch_size=-1, learning_rate=1e-1)

TypeError: train() missing 1 required positional argument: 'Y'

### Plotting the results 

The numerical solution (training set - red, valdiaiton set - green) along with the analytical solution (blue).

In [ ]:
Xe1plot = np.arange(0,1.5, 0.01)
Xe1plot = Xe1plot.reshape((Xe1plot.shape[0], 1, 1))
Ye1 = np.array([example1_trial_solution.predict(Xe1plot[i]) for i in range(Xe1plot.shape[0])]).reshape((Xe1plot.shape[0],))
Xe1plot = Xe1plot.reshape((Xe1plot.shape[0],))
plt.scatter(Xe1plot, Ye1, c='g', label='Numerical - validation', marker='x', s=1)
psi_e1 = lambda x:  np.exp(-0.5*x**2)/(1+x+x**3) + x**2
Ye12 = np.array([psi_e1(Xe1plot[i]) for i in range(Xe1plot.shape[0])])
plt.plot(Xe1plot, Ye12, c='b', label='Analytic')
Ye1sol = np.array([example1_trial_solution.predict(Xe1[i]) for i in range(Xe1.shape[0])]).reshape((Xe1.shape[0],))
plt.scatter(Xe1.reshape((Xe1.shape[0],)), Ye1sol, c='r', label='Numerical - Training', marker='+', s=30)
plt.legend()
plt.show()

In [ ]:
Ye1_interpolation_predict = np.array([example1_trial_solution.predict(Xe1_interpolation[i]) for i in range(Xe1_interpolation.shape[0])]).reshape((Xe1_interpolation.shape[0],))
Ye1_interpolation_true = np.array([psi_e1(Xe1_interpolation[i]) for i in range(Xe1_interpolation.shape[0])]).reshape((Xe1_interpolation.shape[0],))
np.abs(Ye1_interpolation_true - Ye1_interpolation_predict).mean()

In [ ]:
np.abs(Ye1sol - np.array([psi_e1(Xe1[i]) for i in range(Xe1.shape[0])]).reshape((Xe1.shape[0],))).mean()

In [ ]:
plt.clf()
plt.scatter(Xe1plot, Ye1, c='xkcd:sky blue', label='Numerical - validation', marker='x', s=300)
plt.scatter(Xe1.reshape((Xe1.shape[0],)), Ye1sol, c='r', label='Numerical - Training', marker='+', s=1000)
plt.plot(Xe1plot, Ye12, c='xkcd:goldenrod', label='Analytic', linewidth=5)
plt.xlabel(r'$x$', fontsize='50')
plt.ylabel(r'$y$', fontsize='50')
plt.xlim((0,1.5))
plt.ylim((0.7,2.2))
# plt.axis('equal')
plt.legend(fontsize='40')
plt.title('Example 1', fontsize='60')
plt.gcf().set_size_inches(30, 22.5)
plt.tick_params(axis='both', which='major', labelsize=35)
plt.savefig('plots/example1.jpg')
plt.show()

## Example 2

$\frac{d}{dx}\Psi+\frac{1}{5}\Psi=\exp(-\frac{x}{5})\cos(x)$

With boundary initial condition $\Psi(0)=0$ and domain $x\in[0,2]$

In [ ]:
Xe2_interpolation = np.linspace(0,2,num=1000)
Xe2_interpolation = Xe2_interpolation.reshape((Xe2_interpolation.shape[0],1,1))
Xe2 = Xe2_interpolation[::100]
Xe2_interpolation = search(Xe2_interpolation, Xe2)
Xe2_interpolation = Xe2_interpolation.reshape((Xe2_interpolation.shape[0],1,1))

The trial solution for this case is $\Psi(x)=x N(x)$.
The first function below is the function $A(x)=0$
and the second function is the function $B(x)=x$.

In [ ]:
def example2_initial_value(point):
  return 0

In [ ]:
def example2_boundary_vanishing(point):
  return point

### Defining the loss function for a single point and a whole set

The loss function is based on the formula:
$$Loss(N)=\sum_i \left(L\Psi(x_i, N(x_i))-f(x_i,\Psi(x_i, N(x_i))) \right)^2$$
Where N(x) is the neural network and L is some differential operator.

In [ ]:
def example2_loss_function_single_point(self, point, non_squared=False, *kwargs):
  N = self.forward_pass(point, 0)
  dN = self.forward_pass(point, 1)
  loss = (
      point * dN + N + 0.2 * point * N - np.exp(-0.2*point)*np.cos(point)
    )
  if not non_squared:
    loss = loss ** 2
  return loss[0,0]

In [ ]:
def example2_loss_function(self, samples, *kwargs):
  loss = 0
  for i in range(samples.shape[0]):
    loss += self.loss_function_single_point(self, samples[i])
  return loss/samples.shape[0]

### Defining the update rules

The following functions represent $\frac{\partial Loss}{\partial \vec{b}}$, $\frac{\partial Loss}{\partial H}$, and $\frac{\partial Loss}{\partial V}$

In [ ]:
def example2_bias_change(self, point, label, *kwargs):
  db = np.zeros((self.hidden_dim, 1)).astype(dtype="float64")
  loss_sqrt = self.loss_function_single_point(self, point, non_squared=True)
  db_N = self.network_derivative_bias(point, 0)
  db_DN = self.network_derivative_bias(point, 1)
  point = point.reshape((1,))
  for m in range(self.hidden_dim):
    db[m] += 2 * loss_sqrt * ( point * db_DN[0, 0, m] + db_N[0, 0, m] + 0.2 * point * db_N[0, 0, m])
  return db

In [ ]:
def example2_hidden_weights_change(self, point, *kwargs):
  dH = np.zeros((self.hidden_dim, self.input_dim)).astype(dtype="float64")
  loss_sqrt = self.loss_function_single_point(self, point, non_squared=True)
  dH_N = self.network_derivative_hidden_weights(point, 0)
  dH_DN = self.network_derivative_hidden_weights(point, 1)
  for m in range(self.hidden_dim):
    for p in range(self.input_dim):
      dH[m, p] += 2 * loss_sqrt * ( point * dH_DN[0, 0, m, p] + dH_N[0, 0, m, p] + 0.2 * point * dH_N[0, 0, m, p])
  return dH

In [ ]:
def example2_visible_weights_change(self, point, *kwargs):
  dV = np.zeros((self.visible_dim, self.hidden_dim)).astype(dtype="float64")
  loss_sqrt = self.loss_function_single_point(self, point, non_squared=True)
  dV_N = self.network_derivative_visible_weights(point, 0)
  dV_DN = self.network_derivative_visible_weights(point, 1)
  for m in range(self.visible_dim):
    for p in range(self.hidden_dim):
      dV[m, p] += 2 * loss_sqrt * (point * dV_DN[0, 0, m, p] + dV_N[0, 0, m, p] + 0.2 * point * dV_N[0, 0, m, p])
  return dV

### Defining the trial solution with an apropiate network

In [ ]:
example2_trial_solution = nnde.TrialSolution(loss_function=example2_loss_function,
                                        loss_function_single_point=example2_loss_function_single_point,
                                        bias_change=example2_bias_change,
                                        hidden_weights_change=example2_hidden_weights_change,
                                        visible_weights_change=example2_visible_weights_change,
                                        boundary_condition_value_function=example2_initial_value,
                                        boundary_vanishing_function=example2_boundary_vanishing,
                                        input_dim=1, hidden_dim=10, output_dim=1, learning_rate=1e-1, momentum=1e-1)

### Training

In [ ]:
example2_trial_solution.train(Xe2, 10000)

### Plotting the results 

The numerical solution (training set - red, valdiaiton set - green) along with the analytical solution (blue).

In [ ]:
Ye2sol = np.array([example2_trial_solution.predict(Xe2[i]) for i in range(Xe2.shape[0])]).reshape((Xe2.shape[0],))
plt.scatter(Xe2.reshape((Xe2.shape[0],)), Ye2sol, c='r', label='Numerical - Training', marker='+', s=30)
psi_e2 = lambda x: np.exp(-0.2*x) * np.sin(x)
Xe2plot = np.arange(0,2.5, 0.01)
Xe2plot = Xe2plot.reshape((Xe2plot.shape[0], 1, 1))
Ye2 = np.array([example2_trial_solution.predict(Xe2plot[i]) for i in range(Xe2plot.shape[0])]).reshape((Xe2plot.shape[0],))
Xe2plot = Xe2plot.reshape((Xe2plot.shape[0],))
Ye22 = np.array([np.exp(-0.2*Xe2plot[i]) * np.sin(Xe2plot[i]) for i in range(Xe2plot.shape[0])])
plt.scatter(Xe2plot, Ye2, c='g', label='Numerical- Validation', marker='x', s=1)
plt.plot(Xe2plot, Ye22, c='b', label='Analytic')
plt.legend()
plt.show()

In [ ]:
Ye2_interpolation_predict = np.array([example2_trial_solution.predict(Xe2_interpolation[i]) for i in range(Xe2_interpolation.shape[0])]).reshape((Xe2_interpolation.shape[0],))
Ye2_interpolation_true = np.array([psi_e2(Xe2_interpolation[i]) for i in range(Xe2_interpolation.shape[0])]).reshape((Xe2_interpolation.shape[0],))
np.abs(Ye2_interpolation_true - Ye2_interpolation_predict).mean()

In [ ]:
np.abs(Ye2sol - np.array([psi_e2(Xe2[i]) for i in range(Xe2.shape[0])]).reshape((Xe2.shape[0],))).mean()

In [ ]:
plt.clf()
plt.scatter(Xe2plot, Ye2, c='xkcd:sky blue', label='Numerical - validation', marker='x', s=300)
plt.scatter(Xe2.reshape((Xe2.shape[0],)), Ye2sol, c='r', label='Numerical - Training', marker='+', s=1000)
plt.plot(Xe2plot, Ye22, c='xkcd:goldenrod', label='Analytic', linewidth=5)
plt.xlabel(r'$x$', fontsize='50')
plt.ylabel(r'$y$', fontsize='50')
plt.xlim((0,2.5))
plt.ylim((0.,1))
# plt.axis('equal')
plt.legend(fontsize='40')
plt.title('Example 2', fontsize='60')
plt.gcf().set_size_inches(30, 22.5)
plt.tick_params(axis='both', which='major', labelsize=35)
plt.savefig('plots/example2.jpg')
plt.show()

## Example 3

$\frac{d^2}{dx^2}\Psi+\frac{1}{5}\frac{d}{dx}\Psi+\Psi=-\frac{1}{5}\exp(-\frac{x}{5})\cos(x)$

With boundary initial condition $\Psi(0)=0$, $\frac{d}{dx}\Psi(0)=1$ and domain $x\in[0,2]$

Interpolation set.

In [ ]:
Xe3_interpolation = np.linspace(0,2,num=1000)
Xe3_interpolation = Xe3_interpolation.reshape((Xe3_interpolation.shape[0],1,1))
Xe3 = Xe3_interpolation[::100]
Xe3_interpolation = search(Xe3_interpolation, Xe3)
Xe3_interpolation = Xe3_interpolation.reshape((Xe3_interpolation.shape[0],1,1))

The trial solution for this case is $\Psi(x)=x + x^2N(x)$.
The first function below is the function $A(x)=x$
and the second function is the function $B(x)=x^2$.

In [ ]:
def example3_initial_value(point):
  return point

In [ ]:
def example3_boundary_vanishing(point):
  return point ** 2

### Defining the loss function for a single point and a whole set

The loss function is based on the formula:
$$Loss(N)=\sum_i \left(L\Psi(x_i, N(x_i))-f(x_i,\Psi(x_i, N(x_i))) \right)^2$$
Where N(x) is the neural network and L is some differential operator.

In [ ]:
def example3_loss_function_single_point(self, point, non_squared=False, *kwargs):
  N = self.forward_pass(point, 0)
  dN = self.forward_pass(point, 1)
  d2N = self.forward_pass(point, 2)
  loss = ( 2 * N + 4 * point * dN + point ** 2 * d2N + 0.2 * (1 + 2 * point * N + point ** 2 * dN)
     + point + point ** 2 * N + 0.2 * np.exp(-0.2*point)*np.cos(point)
    )
  if not non_squared:
    loss = loss ** 2
  return loss[0,0]

In [ ]:
def example3_loss_function(self, samples, *kwargs):
  loss = 0
  for i in range(samples.shape[0]):
    loss += self.loss_function_single_point(self, samples[i])
  return loss/samples.shape[0]

### Defining the update rules

The following functions represent $\frac{\partial Loss}{\partial \vec{b}}$, $\frac{\partial Loss}{\partial H}$, and $\frac{\partial Loss}{\partial V}$

In [ ]:
def example3_bias_change(self, point, label, *kwargs):
  db = np.zeros((self.hidden_dim, 1)).astype(dtype="float64")
  loss_sqrt = self.loss_function_single_point(self, point, non_squared=True)
  db_N = self.network_derivative_bias(point, 0)
  db_DN = self.network_derivative_bias(point, 1)
  db_D2N = self.network_derivative_bias(point, 2)
  point = point.reshape((1,))
  for m in range(self.hidden_dim):
    db[m] += 2 * loss_sqrt * ( 2 * db_N[0, 0, m] + 4 * point * db_DN[0, 0, m] + point ** 2 * db_D2N[0, 0, m]
                              + 0.2 * (2 * point * db_N[0, 0, m] + point ** 2 * db_DN[0, 0, m])
                              + point ** 2 * db_N[0, 0, m] 
      )
  return db

In [ ]:
def example3_hidden_weights_change(self, point, *kwargs):
  dH = np.zeros((self.hidden_dim, self.input_dim)).astype(dtype="float64")
  loss_sqrt = self.loss_function_single_point(self, point, non_squared=True)
  dH_N = self.network_derivative_hidden_weights(point, 0)
  dH_DN = self.network_derivative_hidden_weights(point, 1)
  dH_D2N = self.network_derivative_hidden_weights(point, 2)
  for m in range(self.hidden_dim):
    for p in range(self.input_dim):
      dH[m, p] += 2 * loss_sqrt * (2 * dH_N[0, 0, m, p] + 4 * point * dH_DN[0, 0, m, p] + point ** 2 * dH_D2N[0, 0, m, p]
                              + 0.2 * (2 * point * dH_N[0, 0, m, p] + point ** 2 * dH_DN[0, 0, m, p])
                              + point ** 2 * dH_N[0, 0, m, p]
      )
  return dH

In [ ]:
def example3_visible_weights_change(self, point, *kwargs):
  dV = np.zeros((self.visible_dim, self.hidden_dim)).astype(dtype="float64")
  loss_sqrt = self.loss_function_single_point(self, point, non_squared=True)
  dV_N = self.network_derivative_visible_weights(point, 0)
  dV_DN = self.network_derivative_visible_weights(point, 1)
  dV_D2N = self.network_derivative_visible_weights(point, 2)
  for m in range(self.visible_dim):
    for p in range(self.hidden_dim):
      dV[m, p] += 2 * loss_sqrt * (2 * dV_N[0, 0, m, p] + 4 * point * dV_DN[0, 0, m, p] + point ** 2 * dV_D2N[0, 0, m, p]
                              + 0.2 * (2 * point * dV_N[0, 0, m, p] + point ** 2 * dV_DN[0, 0, m, p])
                              + point ** 2 * dV_N[0, 0, m, p]   
      )
  return dV

### Defining the trial solution with an apropiate network

In [ ]:
example3_trial_solution = nnde.TrialSolution(loss_function=example3_loss_function,
                                        loss_function_single_point=example3_loss_function_single_point,
                                        bias_change=example3_bias_change,
                                        hidden_weights_change=example3_hidden_weights_change,
                                        visible_weights_change=example3_visible_weights_change,
                                        boundary_condition_value_function=example3_initial_value,
                                        boundary_vanishing_function=example3_boundary_vanishing,
                                        input_dim=1, hidden_dim=10, output_dim=1, learning_rate=1e-2, momentum=1e-1)

### Training

In [ ]:
example3_trial_solution.train(Xe3, 10000)

### Plotting the results 

The numerical solution (training set - red, valdiaiton set - green) along with the analytical solution (blue).

In [ ]:
Ye3sol = np.array([example3_trial_solution.predict(Xe3[i]) for i in range(Xe3.shape[0])]).reshape((Xe3.shape[0],))
plt.scatter(Xe3.reshape((Xe3.shape[0],)), Ye3sol, c='r', label='Numerical - Training', marker='+', s=30)
Xe3plot = np.arange(0,2.5, 0.01)
Xe3plot = Xe3plot.reshape((Xe3plot.shape[0], 1, 1))
Ye3 = np.array([example3_trial_solution.predict(Xe3plot[i]) for i in range(Xe3plot.shape[0])]).reshape((Xe3plot.shape[0],))
Xe3plot = Xe3plot.reshape((Xe3plot.shape[0],))
psi_e3 = lambda x: np.exp(-0.2*x) * np.sin(x)
Ye33 = np.array([psi_e3(Xe3plot[i]) for i in range(Xe3plot.shape[0])])
plt.scatter(Xe3plot, Ye3, c='g', label='Numerical- Validation', marker='x', s=1)
plt.plot(Xe3plot, Ye33, c='b', label='Analytic')
plt.legend()
plt.show()

In [ ]:
print(example3_trial_solution.network.hidden_layer.bias.min())
print(example3_trial_solution.network.hidden_layer.bias.max())
print(example3_trial_solution.network.hidden_layer.weights.min())
print(example3_trial_solution.network.hidden_layer.weights.max())
print(example3_trial_solution.network.visible_layer.weights.min())
print(example3_trial_solution.network.visible_layer.weights.max())

In [ ]:
Ye3_interpolation_predict = np.array([example3_trial_solution.predict(Xe3_interpolation[i]) for i in range(Xe3_interpolation.shape[0])]).reshape((Xe3_interpolation.shape[0],))
Ye3_interpolation_true = np.array([psi_e3(Xe3_interpolation[i]) for i in range(Xe3_interpolation.shape[0])]).reshape((Xe3_interpolation.shape[0],))
np.abs(Ye3_interpolation_true - Ye3_interpolation_predict).mean()

In [ ]:
np.abs(Ye3sol - np.array([psi_e3(Xe3[i]) for i in range(Xe3.shape[0])]).reshape((Xe3.shape[0],))).mean()

In [ ]:
plt.clf()
plt.scatter(Xe3plot, Ye3, c='xkcd:sky blue', label='Numerical - validation', marker='x', s=300)
plt.scatter(Xe3.reshape((Xe3.shape[0],)), Ye3sol, c='r', label='Numerical - Training', marker='+', s=1000)
plt.plot(Xe3plot, Ye33, c='xkcd:goldenrod', label='Analytic', linewidth=5)
plt.xlabel(r'$x$', fontsize='50')
plt.ylabel(r'$y$', fontsize='50')
plt.xlim((0,2.5))
plt.ylim((0.,0.8))
# plt.axis('equal')
plt.legend(fontsize='40')
plt.title('Example 3', fontsize='60')
plt.gcf().set_size_inches(30, 22.5)
plt.tick_params(axis='both', which='major', labelsize=35)
plt.savefig('plots/example3.jpg')
plt.show()